# Data Science Project SoSe 2024
## Team 07
- Maximilian Hoffmann
- Kilian Kempf
- Daniel Schneider
- Tom Schuck

## Submission of Task 2

In [28]:
import os

import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier

### Data Preparation

In [29]:
DATA_DIR = os.path.join(os.getcwd(), 'data/Instacart')

orders = pd.read_csv(os.path.join(DATA_DIR, 'orders.csv.zip'))
op_prior = pd.read_csv(os.path.join(DATA_DIR, 'order_products__prior.csv.zip'))
op_train = pd.read_csv(os.path.join(DATA_DIR, 'order_products__train.csv.zip'))
tip_train = pd.read_csv(os.path.join(DATA_DIR, 'tip_trainingsdaten1_.csv'))[['order_id', 'tip']]
op = pd.concat([op_prior, op_train])

In [30]:
orders_tip_train = pd.merge(orders, tip_train)
orders_tip_train

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip
0,2539329,1,prior,1,2,8,NaN,False
1,2398795,1,prior,2,3,7,15.0,False
2,473747,1,prior,3,3,12,21.0,False
3,2254736,1,prior,4,4,7,29.0,False
4,431534,1,prior,5,4,15,28.0,False
...,...,...,...,...,...,...,...,...
3214869,2558525,206209,prior,9,4,15,22.0,False
3214870,2266710,206209,prior,10,5,18,29.0,False
3214871,1854736,206209,prior,11,4,10,30.0,False
3214872,626363,206209,prior,12,1,12,18.0,False


### Task 2.1

In [304]:
features = ['order_id', 'user_id', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']
# to be updated
param_grid = {'criterion': ['gini', 'entropy', 'log_loss'],
              'max_depth': [depth for depth in range(1, 10)],
              'min_samples_leaf': [2 ** i for i in range(0, 10)]}
cv = KFold(n_splits=5, shuffle=True, random_state=42)
grid_search_clf = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=cv)

In [305]:
X = orders_tip_train[features]
y = orders_tip_train[['tip']]
grid_search_clf.fit(X, y)

KeyboardInterrupt: 

In [283]:
decision_tree_clf = grid_search_clf.best_estimator_
grid_search_clf.best_params_

{'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 4}

### Task 2.2

##### Estimated accuracy of the decision tree classifier on the test set:

In [31]:
grid_search_clf.best_score_

NameError: name 'grid_search_clf' is not defined

### Task 2.3

In [32]:
most_freq_clf = DummyClassifier(strategy='most_frequent')
scores = cross_val_score(most_freq_clf, X, y, cv=cv)
scores.mean()

NameError: name 'X' is not defined

##### Benefit of decision tree classifier over classifier that predicts most frequent class:

In [33]:
grid_search_clf.best_score_ - scores.mean()

NameError: name 'grid_search_clf' is not defined

### Task 2.4

In [34]:
tip_test = pd.read_csv(os.path.join(DATA_DIR, 'tip_testdaten1_template.csv'))
orders_tip_test = pd.merge(orders, tip_test)
orders_tip_test['tip'] = decision_tree_clf.predict(orders_tip_test[features])
orders_tip_test

NameError: name 'decision_tree_clf' is not defined

In [ ]:
orders_tip_test_csv = orders_tip_test[tip_test.columns].copy()
orders_tip_test_csv.rename(columns={tip_test.columns[0]: ''}, inplace=True)
orders_tip_test_csv

In [ ]:
orders_tip_test_csv.to_csv(os.path.join(DATA_DIR, 'tip_testdaten1.csv'), index=False)

### Task 2.5 a)

In [51]:
first_orders = orders[orders['order_number'] == 1]
orders_21137 = op[op['product_id'] == 21137]
first_orders_21137 = first_orders[first_orders['order_id'].isin(orders_21137['order_id'])]
first_orders_21137

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
1159,2669994,77,prior,1,2,11,NaN
1518,185725,101,prior,1,2,11,NaN
1598,2313085,108,prior,1,0,13,NaN
1823,1747819,126,prior,1,6,11,NaN
1943,3204695,135,prior,1,3,19,NaN
...,...,...,...,...,...,...,...
3420524,2962368,206178,prior,1,3,10,NaN
3420593,31169,206183,prior,1,0,13,NaN
3420750,3395125,206194,prior,1,1,9,NaN
3420925,1438269,206204,prior,1,1,11,NaN


##### Percentage of orders containing product 21137 (organic strawberries) first orders of the respective user:

In [52]:
(first_orders_21137.shape[0] / first_orders.shape[0]) * 100

7.984132603329631

### Task 2.5 b)

In [53]:
last_orders = orders[['user_id', 'order_number']].groupby('user_id').max().reset_index()
last_orders = pd.merge(orders, last_orders, on=['user_id', 'order_number'])
last_orders_21137 = last_orders[last_orders['order_id'].isin(orders_21137['order_id'])]
last_orders_21137

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
17,2461523,18,train,7,6,9,7.0
26,2614670,27,train,82,5,14,4.0
58,1651215,59,train,11,0,12,10.0
62,2989905,63,train,40,6,8,13.0
78,21708,79,train,8,0,6,30.0
...,...,...,...,...,...,...,...
206152,642502,206153,train,14,0,8,21.0
206173,1620607,206174,train,53,4,22,10.0
206187,3367945,206188,train,8,1,21,30.0
206199,943915,206200,train,24,6,19,6.0


##### Percentage of orders containing product 21137 (organic strawberries) last orders of the respective user:

In [54]:
(last_orders_21137.shape[0] / last_orders.shape[0]) * 100

5.282989588233297

##### Comparison:
Percentage of orders containing product 21137 (organic strawberries) first orders of the respective user: $\approx 7.984\%$\
Percentage of orders containing product 21137 (organic strawberries) last orders of the respective user: $\approx 5.283\%$
Therefore, the ratio of first orders containing product 21137 is $\approx 51.126\%$\ larger than the ratio of last orders containing product 21137. 
